In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
model.save_pretrained('/content/drive/MyDrive/bert_model_checkpoint')
tokenizer.save_pretrained('/content/drive/MyDrive/bert_model_checkpoint')
torch.save(optimizer.state_dict(), '/content/drive/MyDrive/optimizer_checkpoint.pth')


NameError: name 'torch' is not defined

In [5]:
!pip install transformers datasets


In [6]:
!pip install transformers datasets torch scikit-learn nltk


# Load and Explore the SQuAD Dataset

In [7]:
from datasets import load_dataset

# Load SQuAD
squad = load_dataset("squad")


Inspect the Dataset:

In [8]:
# Check splits
print(squad)

# Preview a sample from the training set
print(squad["train"][0])

# Display dataset features
print(squad["train"].features)


DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 87599
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 10570
    })
})
{'id': '5733be284776f41900661182', 'title': 'University_of_Notre_Dame', 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome

Count the number of examples in each split.

In [9]:
print(f"Training examples: {len(squad['train'])}")
print(f"Validation examples: {len(squad['validation'])}")


Training examples: 87599
Validation examples: 10570


Analyze the lengths of context and question fields:

In [10]:
from collections import Counter

# Length of contexts
context_lengths = [len(example["context"]) for example in squad["train"]]
print(f"Max context length: {max(context_lengths)}")
print(f"Average context length: {sum(context_lengths) / len(context_lengths)}")

# Length of questions
question_lengths = [len(example["question"]) for example in squad["train"]]
print(f"Max question length: {max(question_lengths)}")
print(f"Average question length: {sum(question_lengths) / len(question_lengths)}")


Max context length: 3706
Average context length: 754.3642164864896
Max question length: 25651
Average question length: 59.57113665681115


# Preprocessing the Dataset

mport Tokenizer: We'll use a tokenizer from Hugging Face's transformers library.

In [11]:
from transformers import AutoTokenizer

# Load a pretrained tokenizer (e.g., BERT)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


Tokenize the Dataset: Tokenize the context, question, and align the answers. The answers must be converted to token indices based on the tokenized context.

In [12]:
def preprocess_function(batch):
    tokenized_examples = tokenizer(
        batch["context"],
        batch["question"],
        truncation=True,
        padding="max_length",
        max_length=384,
        return_offsets_mapping=True,  # Helps align tokens with the original text
    )

    start_positions = []
    end_positions = []

    for i, offsets in enumerate(tokenized_examples["offset_mapping"]):
        # Extract the start and end character positions of the answer
        start_char = batch["answers"][i]["answer_start"][0]
        end_char = start_char + len(batch["answers"][i]["text"][0])

        # Initialize token positions
        start_token_idx = end_token_idx = 0

        # Align character positions to token positions
        for idx, (start, end) in enumerate(offsets):
            if start <= start_char < end:
                start_token_idx = idx
            if start < end_char <= end:
                end_token_idx = idx
                break

        # Append token positions for the answer
        start_positions.append(start_token_idx)
        end_positions.append(end_token_idx)

    # Update tokenized examples with start and end positions
    tokenized_examples["start_positions"] = start_positions
    tokenized_examples["end_positions"] = end_positions

    # Remove the offset mapping to save memory
    tokenized_examples.pop("offset_mapping")

    return tokenized_examples


Apply Preprocessing to the Dataset: Use map to apply the preprocessing function to the entire dataset efficiently.

In [13]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)


Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Verify Preprocessing: Inspect the tokenized dataset to ensure it includes all necessary fields.



In [14]:
print(tokenized_squad["train"][0])


{'input_ids': [101, 6549, 2135, 1010, 1996, 2082, 2038, 1037, 3234, 2839, 1012, 10234, 1996, 2364, 2311, 1005, 1055, 2751, 8514, 2003, 1037, 3585, 6231, 1997, 1996, 6261, 2984, 1012, 3202, 1999, 2392, 1997, 1996, 2364, 2311, 1998, 5307, 2009, 1010, 2003, 1037, 6967, 6231, 1997, 4828, 2007, 2608, 2039, 14995, 6924, 2007, 1996, 5722, 1000, 2310, 3490, 2618, 4748, 2033, 18168, 5267, 1000, 1012, 2279, 2000, 1996, 2364, 2311, 2003, 1996, 13546, 1997, 1996, 6730, 2540, 1012, 3202, 2369, 1996, 13546, 2003, 1996, 24665, 23052, 1010, 1037, 14042, 2173, 1997, 7083, 1998, 9185, 1012, 2009, 2003, 1037, 15059, 1997, 1996, 24665, 23052, 2012, 10223, 26371, 1010, 2605, 2073, 1996, 6261, 2984, 22353, 2135, 2596, 2000, 3002, 16595, 9648, 4674, 2061, 12083, 9711, 2271, 1999, 8517, 1012, 2012, 1996, 2203, 1997, 1996, 2364, 3298, 1006, 1998, 1999, 1037, 3622, 2240, 2008, 8539, 2083, 1017, 11342, 1998, 1996, 2751, 8514, 1007, 1010, 2003, 1037, 3722, 1010, 2715, 2962, 6231, 1997, 2984, 1012, 102, 2000, 3183

Save Tokenized Dataset

In [15]:
tokenized_squad.save_to_disk("tokenized_squad")


Saving the dataset (0/1 shards):   0%|          | 0/87599 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10570 [00:00<?, ? examples/s]

# Model Development

Install Necessary Libraries

In [16]:
pip install transformers torch datasets


Load Pre-trained Model and Tokenizer

In [17]:
from transformers import BertForQuestionAnswering, BertTokenizer

# Load smaller model (bert-base-uncased)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Prepare the Dataset for Training

In [19]:
import torch
from torch.utils.data import Dataset

class SquadDataset(Dataset):
    def __init__(self, tokenized_data):
        self.tokenized_data = tokenized_data

    def __len__(self):
        return len(self.tokenized_data)

    def __getitem__(self, idx):
        item = self.tokenized_data[idx]
        input_ids = torch.tensor(item['input_ids'])
        attention_mask = torch.tensor(item['attention_mask'])
        start_positions = torch.tensor(item['start_positions'])
        end_positions = torch.tensor(item['end_positions'])

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'start_positions': start_positions,
            'end_positions': end_positions
        }

# Example of creating a Dataset from the tokenized data
train_dataset = SquadDataset(tokenized_squad["train"])


Set Up Training Configuration

In [ ]:
from transformers import AdamW
from torch.utils.data import DataLoader

# Set up DataLoader for batching
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# Set up optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Set up device (GPU if available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Training loop
model.train()
for epoch in range(3):  # Adjust epochs as needed
    for batch in train_dataloader:
        optimizer.zero_grad()

        # Move batch to device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask,
                         start_positions=start_positions, end_positions=end_positions)

        # Compute loss
        loss = outputs.loss
        loss.backward()

        # Update weights
        optimizer.step()

    # Print loss after each epoch
    print(f"Epoch {epoch + 1}: Loss {loss.item()}")

    # Save model and optimizer after each epoch
    model.save_pretrained('/content/drive/MyDrive/bert_model_checkpoint')
    tokenizer.save_pretrained('/content/drive/MyDrive/bert_model_checkpoint')
    torch.save(optimizer.state_dict(), '/content/drive/MyDrive/optimizer_checkpoint.pth')



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


When you restart your session, you can load files from Google Drive:

In [ ]:
model = BertForQuestionAnswering.from_pretrained('/content/drive/MyDrive/bert_model_checkpoint')
tokenizer = BertTokenizer.from_pretrained('/content/drive/MyDrive/bert_model_checkpoint')
optimizer.load_state_dict(torch.load('/content/drive/MyDrive/optimizer_checkpoint.pth'))


In [ ]:
function ClickConnect(){
    console.log("Clicked on connect button");
    document.querySelector("colab-connect-button").click();
}
setInterval(ClickConnect, 60000);
